# Preparing traning data sets from live camera 

**By Ahmed Hammad**
**ahammad115566@gmail.com**

In [ ]:
import cv2
import time 
def generate_data():
    face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    def face_cropped(img):
        gray_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces_cropped = face_classifier.detectMultiScale(gray_img,1.3,6)
        if face_cropped ==():
            return None 
        for (x,y,w,h) in faces_cropped:
            cropped_faces = img[y:y+h,x:x+w]
            return cropped_faces
    cap = cv2.VideoCapture(0)
    img_id = 0
    while True :
        ret, frame = cap.read()
        if face_cropped(frame) is not None:
            time.sleep(0.1)
            img_id+=1
            face= cv2.resize(face_cropped(frame),(500,500))
            face=cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
            file_path= "pix/Ahmed/"+'Ahmed.'+str(img_id)+'.jpg'
            cv2.imwrite(file_path,face)
            cv2.putText(face,'Id='+str(img_id),(50,50),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
            cv2.imshow('cropped face', face)
            #cv2.waitKey(0)
            if cv2.waitKey(1) & int(img_id) == 500:
                break
    
    cap.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    print('preparing the data set completed....')
        
generate_data()  

# Generate labels from the captured pictures 

In [ ]:
import numpy as np
from tqdm import tqdm 
from random import shuffle 
import os
import cv2
def generate_labels(img_name):
    name = img_name.split('.')[0]
    if name =='Ahmed':
        return np.array([1,0])
    if name=='User2':
        return np.array([0,1])
data=[]
for img in tqdm(os.listdir("pix/data/")):
    path= os.path.join('pix/data/',img)
    img_data=cv2.imread(path,cv2.COLOR_BGR2GRAY)
    img_data=cv2.resize(img_data,(500,500))
    data.append([np.array(img_data),generate_labels(img)])
shuffle(data)      

# Split the data into train and validation samples 

In [ ]:
import tensorflow as tf
train=data[:800]
test=data[800:]
X_train= np.array([i[0] for i in train]).reshape(-1,500,500,1)
Y_train= np.array([i[1] for i in train])
x_test= np.array([i[0] for i in test]).reshape(-1,500,500,1)
y_test= np.array([i[1] for i in test])

# plot the traning sample with labels 

In [ ]:
import matplotlib.pyplot as plt
fig=plt.figure(figsize=(35,27))
for i in range(100):
    
    y=fig.add_subplot(10,10,i+1)    
    y.imshow(X_train[i].reshape(500,500),cmap=plt.cm.gray,interpolation='nearest');
    if Y_train[i][0] == 1:
        plt.title('Ahmed',color='r',fontsize=20)
    if Y_train[i][0] == 0:
        plt.title('User2',color='r',fontsize=20)    
    plt.axis("off")    
   # plt.show()
    

# The model 

In [ ]:
import tensorflow as tf
import tflearn 
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout,fully_connected
from tflearn.layers.estimator import regression

tf.compat.v1.reset_default_graph()
convnet=input_data(shape=[500,500,1])
convnet=conv_2d(convnet,32,5,activation='relu')
convnet=max_pool_2d(convnet,5)
convnet=conv_2d(convnet,64,5,activation='relu')
convnet=max_pool_2d(convnet,5)
convnet=conv_2d(convnet,128,5,activation='relu')
convnet=max_pool_2d(convnet,5)
convnet=conv_2d(convnet,64,5,activation='relu')
convnet=max_pool_2d(convnet,5)
convnet=conv_2d(convnet,32,5,activation='relu')
convnet=max_pool_2d(convnet,5)

convnet=fully_connected(convnet,1024,activation='relu')
convnet=dropout(convnet,0.8)
convnet=fully_connected(convnet,2,activation='softmax')
convnet=regression(convnet,optimizer='adam',learning_rate=0.001,loss='categorical_crossentropy')
model=tflearn.DNN(convnet,tensorboard_verbose=1)
history =model.fit(X_train,Y_train,n_epoch=5,validation_set=(x_test,y_test),show_metric=True,run_id='FRS')

# Make predictions with the validation sample

In [ ]:
import matplotlib.pyplot as plt
fig=plt.figure(figsize=(25,17))
for i in range(100):
    
    y=fig.add_subplot(10,10,i+1)    
    y.imshow(x_test[i].reshape(500,500),cmap=plt.cm.gray,interpolation='nearest');
    if (model.predict([x_test[i]]).flatten()[0])> 0.9:
        plt.title('Ahmed',color='r',fontsize=20)
    if (model.predict([x_test[i]]).flatten()[0]) < 0.01:
        plt.title('User2',color='r',fontsize=20) 
    
    plt.axis('off')   

# Make prediction with Live camera 

In [ ]:
import cv2
import time 
import numpy as np 
import matplotlib.pyplot as plt
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
def face_cropped(img):
    gray_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces_cropped = face_classifier.detectMultiScale(gray_img,1.3,6)
    if face_cropped ==():
         return None 
    for (x,y,w,h) in faces_cropped:
        cropped_faces = img[y:y+h,x:x+w]
        return cropped_faces
cap = cv2.VideoCapture(0)
while True:
    
    ret,frame=cap.read()
    faces_cropped = face_classifier.detectMultiScale(frame,1.3,6)
    for (x,y,w,h) in faces_cropped:
        img=cv2.rectangle(frame,(x,y),(x+w,y+h), (0,255,0),2)
        face= cv2.resize(img[y:y+h,x:x+w],(500,500))
        face=cv2.cvtColor(face,cv2.COLOR_BGR2GRAY)
        face=np.array(face).reshape(-1,500,500,1)
        
        pred=model.predict(face).flatten()[1];
        #print(model.predict(face))
        if pred > 0.5:
            cv2.putText(frame,'Ahmed  '+ '%.2f'%(pred*100)+'%',(x,y-20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)
        elif pred < 0.01:
            cv2.putText(frame,'User2   '+ '%.0f'%((1-pred)*100)+'%',(x,y-20),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),3)    
    cv2.imshow('Frame',frame)
    if cv2.waitKey(1)& 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)